In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.base import ClusterMixin
from sklearn.base import BaseEstimator
sns.set(rc={'figure.figsize':(11.7,8.27)})
from random import *
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.manifold import MDS
from scipy.spatial.distance import cdist

In [11]:
runner = pd.read_csv("../../../output/data_clean/cleaned_runner.csv", index_col=0)
race = pd.read_csv("../../../output/data_clean/cleaned_race_2.csv", index_col= 0)
runner.age_range = pd.Categorical(
    runner.age_range, categories=['16-25','26-40','40-60','61+','Undefined'],ordered=True
)
runner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130137 entries, 0 to 137802
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   race_year_id     130137 non-null  int64   
 1   ranking          113523 non-null  float64 
 2   runner           130137 non-null  object  
 3   time             113523 non-null  object  
 4   gender           130108 non-null  object  
 5   nationality      130137 non-null  object  
 6   time_in_seconds  113523 non-null  float64 
 7   speed            113523 non-null  float64 
 8   runner_id        130108 non-null  object  
 9   age_course       128764 non-null  float64 
 10  age_range        130137 non-null  category
dtypes: category(1), float64(4), int64(1), object(5)
memory usage: 11.0+ MB


In [12]:
runner.ranking = join.ranking * 100 / join.participants

In [13]:
one_hot_gender = pd.get_dummies(runner['gender'], prefix = "gender")
runner = runner.drop('gender',axis = 1)
runner = runner.join(one_hot_gender)
runner = runner.dropna()

In [14]:
join = pd.merge(runner, race, left_on="race_year_id", right_on="race_year_id", how="left")
join = join.loc[join.speed < 20]
join = join.loc[join.distance > 150]

In [15]:
join

,race_year_id,ranking,runner,time,nationality,time_in_seconds,speed,runner_id,age_course,age_range,...,participation,distance,elevation_gain,elevation_loss,aid_stations,participants,year,month,day,frequency_in_ranking
0,68140,0.666667,VERHEUL Jasper,26H 35M 25S,GBR,95725.0,6.276730,VERHEULJasperGBRM30.0,30.0,26-40,...,solo,166.9,4520.0,-4520.0,10,150,2021,9,3,25
1,68140,1.333333,MOULDING JON,27H 0M 29S,GBR,97229.0,6.179638,MOULDINGJONGBRM43.0,43.0,40-60,...,solo,166.9,4520.0,-4520.0,10,150,2021,9,3,25
2,68140,2.000000,RICHARDSON Phill,28H 49M 7S,GBR,103747.0,5.791396,RICHARDSONPhillGBRM38.0,38.0,26-40,...,solo,166.9,4520.0,-4520.0,10,150,2021,9,3,25
3,68140,2.666667,DYSON Fiona,30H 53M 37S,GBR,111217.0,5.402412,DYSONFionaGBRW55.0,55.0,40-60,...,solo,166.9,4520.0,-4520.0,10,150,2021,9,3,25
4,68140,3.333333,FRONTERAS Karen,32H 46M 21S,GBR,117981.0,5.092684,FRONTERASKarenGBRW48.0,48.0,40-60,...,solo,166.9,4520.0,-4520.0,10,150,2021,9,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93700,2871,91.304348,ANANDY Didier,36H 12M 40S,FRA,130360.0,4.664314,ANANDYDidierFRAM46.0,37.0,26-40,...,solo,168.9,10982.0,-10952.0,0,1363,2012,10,18,1363
93701,2871,93.478261,PARMENTIER Sebastien,36H 13M 0S,FRA,130380.0,4.663599,PARMENTIERSebastienFRAM40.0,31.0,26-40,...,solo,168.9,10982.0,-10952.0,0,1363,2012,10,18,1363
93702,2871,95.652174,TALERIEN Thierry,36H 15M 33S,FRA,130533.0,4.658132,TALERIENThierryFRAM53.0,44.0,40-60,...,solo,168.9,10982.0,-10952.0,0,1363,2012,10,18,1363
93703,2871,97.826087,POCHAT Joseph,36H 33M 43S,FRA,131623.0,4.619557,POCHATJosephFRAM47.0,38.0,26-40,...,solo,168.9,10982.0,-10952.0,0,1363,2012,10,18,1363


In [70]:
def get_new_runners_per_year(df):
    years = df.year.unique().tolist()[::-1][1:]
    new_runners = []
    
    for year in years:
        search_df = df.loc[df.year < year]
        current_year_df = df.loc[df.year == year]
        unique_runners = current_year_df.runner_id.unique()
        new_year_runner = 0
        
        for runner in unique_runners:
            print(runner)
            if search_df.loc[search_df.runner_id == runner]:
                new_year_runner += 1
        new_runners.append(new_year_runner)
            
    return years

In [71]:
get_new_runners_per_year(join)

CATMUREdwardGBRM39.0


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().